In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import ArrayType, FloatType, StringType, StructType, StructField
from pyspark.sql.functions import col
from pyspark import SparkFiles, SparkContext

In [ ]:
configs = {
    "spark.app.name": "ResNetPredictNotebook",
    "spark.master": "yarn",                    
    "spark.submit.deployMode": "client",       
    "spark.executor.instances": "2",        
    "spark.executor.cores": "1",
    "spark.executor.memory": "1g",
    "spark.executor.memoryOverhead": "512m",
    "spark.driver.memory": "1.5g",
    "spark.sql.shuffle.partitions": "8",
    "spark.default.parallelism": "8",
    "spark.python.worker.reuse": "true",
    # passa env var agli executor: OMP/MKL/OPENBLAS threads a 1
    "spark.executorEnv.OMP_NUM_THREADS": "1",
    "spark.executorEnv.MKL_NUM_THREADS": "1",
    "spark.executorEnv.OPENBLAS_NUM_THREADS": "1",
}
try:
    spark.stop()
except NameError:
    pass

if SparkContext._active_spark_context is not None:
    SparkContext._active_spark_context.stop()
    
builder = SparkSession.builder
for k, v in configs.items():
    builder = builder.config(k, v)

# crea la session (se non disponibile, la crea)
spark = builder.getOrCreate()
sc = spark.sparkContext
print(dict(sc.getConf().getAll()))

25/08/21 12:14:17 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


{'spark.executorEnv.OMP_NUM_THREADS': '1', 'spark.app.name': 'ResNetPredictNotebook', 'spark.driver.memory': '1.5g', 'spark.python.worker.reuse': 'true', 'spark.default.parallelism': '8', 'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES': 'http://namenode:8088/proxy/application_1755771130202_0001', 'spark.driver.host': 'namenode', 'spark.executor.memory': '1g', 'spark.serializer.objectStreamReset': '100', 'spark.ui.proxyBase': '/proxy/application_1755771130202_0001', 'spark.submit.deployMode': 'client', 'spark.driver.port': '34487', 'spark.ui.filters': 'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter', 'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS': 'namenode', 'spark.driver.appUIAddress': 'http://namenode:4040', 'spark.app.id': 'application_1755771130202_0001', 'spark.executor.memoryOverhead': '512m', 'spark.driver.extraJavaOptions': '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOp

In [4]:
print("Spark master:", sc.master)

Spark master: yarn


In [6]:
import torch
import torchvision.models as models

weights = models.ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)
state_dict = model.state_dict()
torch.save(state_dict, "/tmp/resnet50_statedict2.pth")

sc.addFile("/tmp/resnet50_statedict2.pth")

In [7]:
def make_resnet_fn():
    """
    Questa funzione viene eseguita una volta per python worker (executor).
    Deve restituire una funzione `predict` che riceve un np.ndarray (batch) e ritorna
    un np.ndarray 2D (batch, embedding_dim).
    """
    import io
    import numpy as np
    from PIL import Image
    import torch
    import torch.nn as nn
    import torchvision.models as models
    import torchvision.transforms as T

    #legge i pesi del modello salvati in precedenza in spark
    local_path = SparkFiles.get("resnet50_statedict2.pth")
    weights = models.ResNet50_Weights.DEFAULT
    preprocess = weights.transforms() 
    
    # carica il modello e prepara feature_extractor rimuovendo ultimo layer
    model = models.resnet50(weights=None)
    state_dict = torch.load(local_path, map_location="cpu", weights_only=False)
    model.load_state_dict(state_dict)
    feature_extractor = nn.Sequential(*list(model.children())[:-1])  
    feature_extractor.eval()
    device = "cpu"
    feature_extractor.to(device)

    def predict(content_bytes_arr: np.ndarray) -> np.ndarray:
        """
        content_bytes_arr: 1-d numpy array of bytes-like objects (batch,)
        ritorna: 2-d numpy array float32 shape (batch, 2048)
        """
        # costruisci batch tensor
        imgs = []
        for b in content_bytes_arr:
            try:
                img = Image.open(io.BytesIO(b)).convert("RGB")
            except Exception:
                # se immagine corrotta -> sostituisci con black image (o gestisci come preferisci)
                img = Image.new("RGB", (224,224))
            imgs.append(preprocess(img))

        xs = torch.stack(imgs).to(device)  # shape (batch, 3, 224, 224)
        with torch.no_grad():
            feats = feature_extractor(xs)                     # (batch, 2048, 1, 1)
            feats = feats.reshape(feats.size(0), -1)          # (batch, 2048)
            out = feats.cpu().numpy().astype(np.float32)      # numpy 2D
        return out

    return predict


In [8]:
# Legge file binari (binaryFile): colonne incluse: path, modificationTime, length, content (binary)
df = spark.read.format("binaryFile").load("hdfs:///user/hadoopuser/flickr30k_images/flickr30k_images/").select("path", "content")

#df.show()


25/08/21 12:16:05 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [9]:
df.summary().show()

+-------+--------------------+
|summary|                path|
+-------+--------------------+
|  count|               31784|
|   mean|                NULL|
| stddev|                NULL|
|    min|hdfs://namenode:9...|
|    25%|                NULL|
|    50%|                NULL|
|    75%|                NULL|
|    max|hdfs://namenode:9...|
+-------+--------------------+



In [ ]:
# crea il predict_batch_udf: ritorna una colonna ArrayType(FloatType()) (2D --> array per riga)
resnet_udf = predict_batch_udf(
    make_resnet_fn,
    return_type=ArrayType(FloatType()),   # 2D: (batch, dim) -> ogni riga riceverà un array<float> (embedding)
    batch_size=4
)

# Applichiamo il predict UDF sulla colonna 'content' (bytes)
df_with_emb = df.withColumn("embedding", resnet_udf(col("content"))).select("path", "embedding")

# Scriviamo in Parquet (scalabile)
df_with_emb.write.mode("overwrite").parquet("hdfs:///user/hadoopuser/flickr_image_embeddings_parquet/")

[Stage 3:=>                                                     (36 + 1) / 1012]

In [ ]:
#test con funz predict_batch_udf che calcola solo costanti
'''
def make_constant():
    import numpy as np
    def predict(batch):
        # batch è np.ndarray di oggetti; ritorniamo (B, 4) per semplicità
        B = len(batch)
        out = np.zeros((B,4), dtype=np.float32) + 0.5
        return out
    return predict

const_udf = predict_batch_udf(make_constant, return_type=ArrayType(FloatType()), batch_size=2)
#df.show()
#df = spark.read.format("binaryFile").load("hdfs:///some/small/images/").limit(10).select("path","content")
df2 = df.withColumn("emb", const_udf(col("content")))
df2.show()
'''

In [13]:

# Carica embeddings dal parquet
df_from_parquet = spark.read.parquet("hdfs:///user/hadoopuser/test2_image_embeddings_parquet_test2/")

# Controlla schema
df_from_parquet.printSchema()
# dovrebbe mostrare: path: string, embedding: array<float>
df_from_parquet.show()
df_from_parquet.summary().show()

root
 |-- path: string (nullable = true)
 |-- embedding: array (nullable = true)
 |    |-- element: float (containsNull = true)



+--------------------+--------------------+
|                path|           embedding|
+--------------------+--------------------+
|hdfs://namenode:9...|[0.0038882254, 0....|
|hdfs://namenode:9...|[0.0, 0.0, 0.0, 0...|
|hdfs://namenode:9...|[0.04085762, 0.0,...|
|hdfs://namenode:9...|[0.022724897, 0.0...|
|hdfs://namenode:9...|[0.062641725, 0.0...|
|hdfs://namenode:9...|[1.0650856, 0.003...|
|hdfs://namenode:9...|[0.9971608, 0.0, ...|
|hdfs://namenode:9...|[0.0, 0.0, 0.0, 0...|
|hdfs://namenode:9...|[0.48371467, 0.01...|
|hdfs://namenode:9...|[0.11777613, 0.0,...|
|hdfs://namenode:9...|[0.018268155, 0.0...|
|hdfs://namenode:9...|[0.033272073, 0.0...|
|hdfs://namenode:9...|[0.033873044, 0.0...|
|hdfs://namenode:9...|[0.0, 0.0, 0.0, 0...|
|hdfs://namenode:9...|[0.0, 0.047265064...|
|hdfs://namenode:9...|[0.07706165, 0.0,...|
|hdfs://namenode:9...|[0.03996246, 0.13...|
|hdfs://namenode:9...|[2.1814766, 0.0, ...|
|hdfs://namenode:9...|[0.0, 0.0, 0.0067...|
|hdfs://namenode:9...|[0.2139859

[Stage 13:======================================================> (32 + 1) / 33]

+-------+--------------------+
|summary|                path|
+-------+--------------------+
|  count|               31784|
|   mean|                NULL|
| stddev|                NULL|
|    min|hdfs://namenode:9...|
|    25%|                NULL|
|    50%|                NULL|
|    75%|                NULL|
|    max|hdfs://namenode:9...|
+-------+--------------------+



In [16]:
from pyspark.sql.functions import col, size, expr, sqrt
bad_dim_count = df_from_parquet.filter(size(col("embedding")) != 2048).count()

In [17]:
print(bad_dim_count)

0


In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
from pymilvus import connections, utility, Collection, FieldSchema, CollectionSchema, DataType
import numpy as np

# Configurazione e Creazione della Collezione su Milvus (eseguito sul driver) 

# Parametri di connessione e della collezione
MILVUS_HOST = "192.168.100.4"
MILVUS_PORT = "19530"
COLLECTION_NAME = "image_embeddings_spark"
DIMENSION = 2048 # Dimensione dei vettori ResNet50

# Connessione a Milvus dal driver
print(f"Connessione a Milvus su {MILVUS_HOST}:{MILVUS_PORT}")
connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

# Controlla se la collezione esiste già e, in caso, la elimina per rieseguire lo script
if utility.has_collection(COLLECTION_NAME):
    print(f"Collezione '{COLLECTION_NAME}' esistente. Verrà eliminata e ricreata.")
    utility.drop_collection(COLLECTION_NAME)

# Definisce lo schema della collezione
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="path", dtype=DataType.VARCHAR, max_length=65535), 
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields, description="Image embeddings generated with Spark")

# Crea la collezione
print(f"Creazione della collezione '{COLLECTION_NAME}'...")
collection = Collection(name=COLLECTION_NAME, schema=schema)
print("Collezione creata con successo.")

In [ ]:
# Definizione della Funzione per il Caricamento Distribuito su Milvus

# Questa funzione verrà eseguita su ogni partizione (worker) del DataFrame
def upload_partition_to_milvus(partition):
    # Importa le librerie necessarie all'interno della funzione
    from pymilvus import connections, Collection
    
    # Ogni worker deve stabilire la propria connessione a Milvus
    connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    collection = Collection(name=COLLECTION_NAME)
    
    rows = list(partition)
    if not rows:
        connections.disconnect("default")
        return

    # Estrai i dati in colonne
    ids = [row["id"] for row in rows]
    paths = [row["path"] for row in rows]
    vectors = [row["embedding"] for row in rows]
    
    data_to_insert = [ids, paths, vectors]
    
    # Inserisci i dati della partizione
    collection.insert(data_to_insert)
    
    # Disconnetti il worker
    connections.disconnect("default")

In [ ]:
# Caricamento Distribuito con Spark 

# Aggiunge una colonna con ID univoci al DataFrame letto da Parquet
df_with_ids = df_from_parquet.withColumn("id", monotonically_increasing_id())

# riesegue la cache del df per evitare di ricalcolarlo
df_with_ids.cache()

print("Inizio del caricamento distribuito dei dati su Milvus...")
# Applica la funzione 'upload_partition_to_milvus' a ogni partizione del DataFrame
df_with_ids.foreachPartition(upload_partition_to_milvus)
print("Caricamento dati completato.")

In [ ]:
# Creazione dell'Indice e Caricamento in Memoria (eseguito sul driver) 

# Assicura che tutti i dati siano stati scritti su disco in Milvus
print("Flushing dei dati...")
collection.flush()
print(f"Numero totale di entità nella collezione: {collection.num_entities}")

# Definisce i parametri per l'indice
index_params = {
    "metric_type": "COSINE",       # Metrica di distanza (cosine)
    "index_type": "IVF_FLAT",  # Indice standard per bilanciare velocità e accuratezza
    "params": {"nlist": 1024}  # Numero di cluster, da calibrare in base ai dati
}

# Crea l'indice sul campo vettoriale
print(f"Creazione dell'indice {index_params['index_type']}...")
collection.create_index(field_name="embedding", index_params=index_params)
print("Indice creato con successo.")

# Carica la collezione in memoria per renderla disponibile alle ricerche
print("Caricamento della collezione in memoria...")
collection.load()
print("Collezione caricata e pronta per le ricerche.")

# Disconnettiti dal driver
connections.disconnect("default")

In [ ]:
# VERIFICA che il DB risponda correttamente a una ricerca di similarità

# Connessione a Milvus 
print(f"Connessione a Milvus su {MILVUS_HOST}:{MILVUS_PORT}...")
connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

# Verifica il Numero di Vettori 
print("\n1. Conteggio delle entità")
if not utility.has_collection(COLLECTION_NAME):
    print(f"ERRORE: La collezione '{COLLECTION_NAME}' non è stata trovata.")
else:
    collection = Collection(name=COLLECTION_NAME)
    collection.flush() # Assicura che gli ultimi dati scritti siano indicizzati
    print(f"La collezione '{COLLECTION_NAME}' contiene {collection.num_entities} entità.")

    # Controlla lo Stato della Collezione e dell'Indice 
    print("\n2. Stato della collezione e dell'indice")
    print(f"La collezione ha un indice?")
    print(f"Indici presenti: {collection.indexes}")
    
    # Verifica lo stato di caricamento
    load_state = utility.get_query_segment_info(COLLECTION_NAME)[0].state
    print(f"Stato di caricamento in memoria: {load_state}")


    # Recupera un'Entità di Esempio (Query) 
    print("\n3. Recupero di un'entità di esempio")
    # Recuperiamo l'entità con id = 0 (la prima inserita da Spark)
    sample_entity = collection.query(
      expr = "id == 0",
      output_fields = ["id", "path"]
    )
    if sample_entity:
        print("Dati recuperati per id=0:")
        print(sample_entity)
        sample_path = sample_entity[0]['path']
    else:
        print("Nessuna entità trovata con id=0.")
        sample_path = None

    # Esegui una Ricerca per Similitudine 
    print("\n4. Test di ricerca per similitudine")
    if sample_path:
        # Prendiamo il vettore dell'entità che abbiamo appena recuperato
        vector_to_search = collection.query(expr=f"path == '{sample_path}'", output_fields=["embedding"])[0]['embedding']
        
        # Impostiamo i parametri della ricerca
        search_params = {
            "metric_type": "COSINE",
            "params": {"nprobe": 10}, 
        }

        # Eseguiamo la ricerca
        results = collection.search(
            data=[vector_to_search],          # Il vettore da cercare
            anns_field="embedding",           # Il campo su cui cercare
            param=search_params,
            limit=3,                          # Vogliamo i 3 risultati più simili
            output_fields=["path"]            # Vogliamo ottenere il path delle immagini trovate
        )

        # Mostriamo i risultati
        print("La ricerca ha prodotto i seguenti risultati:")
        for i, hit in enumerate(results[0]):
            print(f"  Risultato {i+1}:")
            print(f"    - ID: {hit.id}")
            print(f"    - Distanza: {hit.distance:.4f} ")
            print(f"    - Path: {hit.entity.get('path')}")
            
        print("\nSe il primo risultato ha distanza 1 e corrisponde all'immagine di partenza, la ricerca funziona")

# Disconnessione 
connections.disconnect("default")

In [18]:
spark.stop()